# Setup

In [1]:
from py2neo import Graph
from py2neo.bulk import create_nodes, create_relationships
import pandas as pd
import numpy as np
import os
import json
from collections import Counter
from itertools import islice, cycle
import spacy
import pytextrank
import time
from pandas import json_normalize

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

2022-05-18 21:48:57.697021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 21:48:57.697064: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
user = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')
uri = 'bolt://localhost:7687'

graph = Graph(uri, auth=(user, password))

## Parameters Setup

In [3]:
# create interest lists
with open("../data/COURSE/go1contents_2604_reg.json", "r") as f:
    courses = json.load(f)

topics = [[y["value"] for y in x.get("attributes", {}).get("topics", [])] for x in courses]
interests_list = list(pd.DataFrame(topics, columns=["level1","level2","level3"])['level2'].value_counts().keys())

# create occupations list
esco_occupations = pd.read_csv("../data/ESCO/occupations_en.csv")
occ = list(esco_occupations['preferredLabel'].drop_duplicates().sort_values()) + ['others']

In [4]:
# create widget for job
def get_other_job_update(other_choice=''):
    return other_choice

def get_job_update(choice=''):    
    if choice == 'others':
        display(other_job)
    else:
        return choice

def get_selected_job(job, other_job):
    if not job.result:
        return other_job.result
    else:
        return job.result
    
other_job = widgets.interactive(get_other_job_update, other_choice='')
job = widgets.interactive(get_job_update, choice=widgets.Combobox(options=occ, disabled=False))


# create widget for interests
interest_w = widgets.TagsInput(allowed_tags= interests_list, allow_duplicates=False)

In [5]:
accordion = widgets.Accordion(children=[job,interest_w], titles=(['Please select your current job title','What are you interested in learning? (choose max 5)']))
accordion

Accordion(children=(interactive(children=(Combobox(value='', description='choice', options=(' food service wor…

## Run query to get the recommended courses based on the intersection of Interest and Job

In [6]:
occ = get_selected_job(job,other_job)
interests = interest_w.value

In [9]:
query = '''

CALL db.index.fulltext.queryNodes('OccupationFT',$occ) YIELD node,score
WITH *
MATCH (node)-[r:REQUIRE]->(s:Skill)-[h:HAS_ESCO_TOPIC]->(e:ESCO_Topic)<-[b:BELONG_TO]-(c:Course)-[hs:HAS]->(t:GO1_Topic)
WHERE t.preferred_label IN $interests
WITH c.uri as course_id, c.language as language, c.duration as duration, CASE c.course_type
  WHEN 'video'  THEN 1
  WHEN 'interactive' THEN 2
  WHEN 'course' THEN 3
  WHEN 'link' THEN 4
  WHEN 'document' THEN 5
  WHEN 'audio' THEN 6
  WHEN 'text' THEN 7
  ELSE 8 END AS c_type_id,
collect({job:node.preferred_label, job_score:score, go1_topic:t.preferred_label})[0] as result ORDER BY result.job_score DESC, result.go1_topic, c_type_id, duration
WITH course_id
MATCH (c:Course) WHERE c.uri IN course_id
RETURN c

'''

In [11]:
cypherResult = graph.run(query, parameters = {'occ':occ, 'interests':interests}).data()

In [12]:
json_normalize(cypherResult)

,c.duration,c.preferred_label,c.course_type,c.description,c.language,c.uri
0,1,Installing sklearn,video,Course Overview Machine Learning is one of...,en,11763485
1,1,Geometrical representation,video,Course Overview Machine Learning is one of...,en,11763495
2,1,Introduction to Regression Analysis,video,Course Overview Machine Learning is one of...,en,11763486
3,1,Introduction to logistic regression,video,Course Overview Machine Learning is one of...,en,11764315
4,1,Using Searborn for graphs,video,Course Overview Machine Learning is one of...,en,11763397
...,...,...,...,...,...,...
17822,3,Gmail - Contact Groups in Gmail,video,Course Overview This 2 minute video cove...,en,13406866
17823,20,Office 2010 - Outlook Intermediate - Working W...,interactive,Click on the New Note button in the New gr...,en,6349950
17824,19,Die Rollen der Gruppe - The roles of the group,interactive,Du bist unsicher und verwirrt hinsichtlich dei...,de,36172731
17825,21,Problem solving - Problem Solving,interactive,Zdarzyło Ci się kiedyś czuć zagrożenie ze stro...,pl,36197446
